In [1]:
import pandas as pd
import numpy as np

In [2]:
from get_3year_data import get_period_data
from process_data import process_wave_data

In [3]:
first_wave_new,second_wave_new,third_wave_new,fourth_wave_new = get_period_data()

Reading data ...
Getting 3 year periods data ...
Number of investgated elders situations :  
               start year observations  survivor  death  missing
2002 to 2005                    16064      8175   5874     2015
2005 to 2008                    15638      7472   5228     2938
2008 to 2011                    16954      8418   5635     2894
2011 to 2014                     9765      6066   2878      820
Selecting needed variables ...


/Users/wangweiren/Desktop/腾讯医疗/CLHLS_master/get_3year_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_data[column] = pd.to_numeric(wave_data[column],errors = 'coerce')
/Users/wangweiren/Desktop/腾讯医疗/CLHLS_master/get_3year_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wave_data[column] = pd.to_numeric(wave_data[column],errors = 'coerce')
/Users/wangweiren/Desktop/腾讯医疗/CLHLS_master/get_3year_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

Filling missing values ...
Missing values situation
f34         0.29%
a43         0.24%
d71         0.10%
d72         0.13%
d81         0.12%
d82         0.20%
d91         0.15%
d92         0.21%
d31         0.02%
d32         0.07%
d11f        0.02%
e1          0.20%
e5          0.04%
e6          0.01%
g15a1       0.01%
g15b1       0.01%
g15c1       0.01%
g15d1       0.01%
g15e1       0.01%
g15f1       0.01%
g15g1       0.01%
g15h1       0.01%
g15i1       0.01%
g15j1       0.01%
g15k1       0.01%
g15l1       0.01%
g15m1       0.01%
g15n1       0.01%
g15o1       0.01%
f34_5       0.72%
a51_5       0.04%
d71_5       0.04%
d72_5       0.13%
d81_5       0.05%
d82_5       0.20%
d92_5       0.21%
d11b_5      0.01%
d11g_5      0.01%
d11h_5      0.01%
e1_5        0.01%
e2_5        0.02%
e3_5        0.01%
e4_5        0.01%
e5_5        0.04%
e6_5        0.02%
g15a1_5     0.01%
g15d1_5     0.01%
g15g1_5     0.01%
g15h1_5     0.05%
g15i1_5     0.09%
g15j1_5    54.23%
g15k1_5     0.11%
g15l1_5     

In [4]:
wave1,wave2,wave3,wave4 = process_wave_data(first_wave_new,second_wave_new,third_wave_new,fourth_wave_new)

Before drop Depression missing :  7271
After drop Depression missing :  6368
Before drop Depression missing :  6592
After drop Depression missing :  5898
Before drop Depression missing :  7337
After drop Depression missing :  6354
Before drop Depression missing :  5408
After drop Depression missing :  4639
Processing invariant variables ...
Done!
Processing invariant variables ...
Done!
Processing invariant variables ...
Done!
Processing invariant variables ...
Done!
Processing variant variables ...
Before drop Dementia :  6368
After drop Dementia :  6278
Done!
Processing variant variables ...
Before drop Dementia :  5898
After drop Dementia :  5830
Done!
Processing variant variables ...
Before drop Dementia :  6354
After drop Dementia :  6248
Done!
Processing variant variables ...
Before drop Dementia :  4639
After drop Dementia :  4564
Done!


## 合并数据成总的df

In [5]:
from get_total_df_for_ML import get_total_df

In [6]:
total_df = get_total_df(wave1,wave2,wave3,wave4)

6278 5830 6248 4564
22920


In [7]:
len(total_df)

22920

## 计算三年间diff变量，获取各组变量包含的变量名

In [8]:
from get_current_diff_var import get_current_diff_variable_list

In [9]:
_,diff_var = get_current_diff_variable_list(total_df)

Creating diff variables based on current variables and 3 years later variables ...
Done!


In [10]:
for var in list(filter(lambda x:True if x[:3]=='std' else False,diff_var))+['hypertension_diff','ADL_diff','Parkinson_diff','agegroup_diff', 'depression_score_diff','diabetes_diff','stoke_CVD_diff','co-habitation_diff', 'economy_status_diff']:
    diff_var.remove(var)

# Logistic Regression

In [11]:
from logistic_regression import logistic_regression

In [12]:
pd.set_option('display.max_rows',200)
pd.set_option('display.float_format',lambda x:'%.3f' %x)

In [13]:
total_df_for_logit = total_df.copy()
for col in total_df_for_logit.columns:
    total_df_for_logit[col] = pd.to_numeric(total_df_for_logit[col],downcast='float')

#### Logistic Regression 2
    X: three year before information
    Y: three year after cognitive impairment

In [14]:
current_variables = ['gender',  'ADL', 'education', 'agegroup', 'std_plant_flower_bird', 'std_read_news_books', 'std_majong_or_cards', 'std_tv_or_radio', 'current_CI_score']

In [15]:
logit_var1 = current_variables+ ['later_CI']
logit_df1 = total_df_for_logit[logit_var1]
cat_vars = ['education','agegroup']
res1 = logistic_regression(logit_df1,'later_CI',cat_vars)

Optimization terminated successfully.
         Current function value: 0.239832
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.333937
         Iterations 8


In [16]:
def get_OR(x):
    return str('%.2f' %x['OddRatio'])+'('+str('%.2f' %x['95%lower']) +','+str('%.2f' %x['95%upper']) + ')'

In [17]:
params1 = res1['params']
params1['OR'] = params1.apply(lambda x:get_OR(x),axis = 1)
output1 = params1[['Variable','Num of samples','OR','pvalues']]
output1

,Variable,Num of samples,OR,pvalues
0,gender,11171,"0.73(0.64,0.82)",0.000
1,ADL,2010,"1.25(1.10,1.44)",0.001
2,std_plant_flower_bird,4326,"0.75(0.63,0.89)",0.001
3,std_read_news_books,6094,"0.80(0.67,0.97)",0.023
4,std_majong_or_cards,1873,"0.69(0.53,0.90)",0.006
5,std_tv_or_radio,18579,"0.72(0.64,0.80)",0.000
6,current_CI_score,3,"0.90(0.89,0.90)",0.000
7,education_1.0,7938,"0.66(0.58,0.77)",0.000
8,education_2.0,3115,"0.60(0.47,0.77)",0.000
9,agegroup_1.0,7105,"2.06(1.78,2.38)",0.000


#### Logistic Regression 2
    X: three year before information + changed behaviors during 3 years
    Y: three year after cognitive impairment

In [18]:
diff_var = ['tv_or_radio_diff','plant_flower_bird_diff','majong_or_cards_diff','read_news_books_diff']

In [19]:
logit_var2 = current_variables + diff_var + ['later_CI']
logit_df2 = total_df_for_logit[logit_var2]
cat_vars2 = cat_vars + ['exercise_diff','eat_fruits_diff','eat_vegs_diff','majong_or_cards_diff','outdoor_activity_diff','plant_flower_bird_diff','raise_domestic_animals_diff','read_news_books_diff','tv_or_radio_diff']
cat_vars2 = list(set(cat_vars2)&set(logit_var2))
res2 = logistic_regression(logit_df2,'later_CI',cat_vars2,standardized_diff = True)

Standardizing diff variables ...
Standardize diff variables done!
Optimization terminated successfully.
         Current function value: 0.229846
         Iterations 9
majong_or_cards_diff_-1.0
read_news_books_diff_-1.0
plant_flower_bird_diff_-1.0
tv_or_radio_diff_-1.0
Optimization terminated successfully.
         Current function value: 0.290907
         Iterations 8


In [20]:
params2 = res2['params']
params2['OR'] = params2.apply(lambda x:get_OR(x),axis = 1)
output2 = params2[['Variable','Num of samples','OR','pvalues']]
output2.set_index('Variable',inplace = True)

In [21]:
output2.loc[list(output1.Variable),:]

,Num of samples,OR,pvalues
Variable,,,
gender,11171,"0.78(0.69,0.88)",0.000
ADL,2010,"1.27(1.11,1.46)",0.001
std_plant_flower_bird,4326,"0.62(0.47,0.82)",0.001
std_read_news_books,6094,"0.25(0.15,0.40)",0.000
std_majong_or_cards,1873,"0.59(0.44,0.79)",0.000
std_tv_or_radio,18579,"0.42(0.36,0.48)",0.000
current_CI_score,3,"0.91(0.90,0.91)",0.000
education_1.0,7938,"0.71(0.62,0.82)",0.000
education_2.0,3115,"0.79(0.61,1.03)",0.079


In [22]:
output2.loc[list(set(output2.index)-set(output1.Variable)),:].sort_values(by = 'Variable')

,Num of samples,OR,pvalues
Variable,,,
majong_or_cards_diff_-1.0,3341,"1.27(1.06,1.51)",0.009
majong_or_cards_diff_1.0,2629,"0.49(0.38,0.64)",0.000
plant_flower_bird_diff_-1.0,3315,"1.36(1.04,1.77)",0.026
plant_flower_bird_diff_1.0,3200,"0.54(0.43,0.68)",0.000
read_news_books_diff_-1.0,3166,"4.18(2.55,6.83)",0.000
read_news_books_diff_1.0,2555,"0.79(0.61,1.03)",0.085
tv_or_radio_diff_-1.0,5503,"2.27(1.99,2.60)",0.000
tv_or_radio_diff_1.0,4811,"0.67(0.59,0.77)",0.000


In [23]:
diff_variables_pos = list(filter(lambda x:True if 'diff' in x and x[-4:] == '_1.0' else False,list(output2.index)))

In [24]:
significant_pos_diff = output2.loc[diff_variables_pos,:].query('pvalues<0.05')
significant_pos_diff

,Num of samples,OR,pvalues
Variable,,,
majong_or_cards_diff_1.0,2629,"0.49(0.38,0.64)",0.000
plant_flower_bird_diff_1.0,3200,"0.54(0.43,0.68)",0.000
tv_or_radio_diff_1.0,4811,"0.67(0.59,0.77)",0.000


In [25]:
need_divide_diff = list(map(lambda x:x[:-4],list(significant_pos_diff.index)))
if 'exercise_diff' in need_divide_diff:
    need_divide_diff.remove('exercise_diff')

#### Logistic Regression 3
    X: three year before information + detailed changed behaviors during 3 years
    Y: three year after cognitive impairment

In [26]:
def divide_significant_diff(x):
    return x if x>=0 else -1

In [27]:
def divide_significant_diff(x):
    if x < 0:return -1
    if x == 0:return 0
    if x == 1:return 1
    if x == 2:return 1
    if x == 3:return 2
    if x == 4:return 2

In [28]:
logit_df3 = logit_df2.copy()
for var in need_divide_diff:
    logit_df3[var] = logit_df3[var].apply(lambda x:divide_significant_diff(x))

In [29]:
var3 = need_divide_diff + current_variables
var3.append('later_CI')
logit_df3 = logit_df3.loc[:,var3]
cat_vars3 = [ 'education', 'agegroup'] + need_divide_diff

In [30]:
for var in need_divide_diff:
    logit_df3 = logit_df3[logit_df3[var]!=5]

In [31]:
res4 = logistic_regression(logit_df3,'later_CI',cat_vars3)

Optimization terminated successfully.
         Current function value: 0.230329
         Iterations 8
majong_or_cards_diff_-1.0
plant_flower_bird_diff_-1.0
tv_or_radio_diff_-1.0
Optimization terminated successfully.
         Current function value: 0.295517
         Iterations 8


In [32]:
params3 = res4['params']
params3['OR'] = params3.apply(lambda x:get_OR(x),axis = 1)
output3 = params3[['Variable','Num of samples','OR','pvalues']]
output3.set_index('Variable',inplace = True)

In [33]:
output3.loc[list(output1.Variable),:]

,Num of samples,OR,pvalues
Variable,,,
gender,11171,"0.76(0.68,0.86)",0.000
ADL,2010,"1.29(1.13,1.48)",0.000
std_plant_flower_bird,4326,"0.58(0.45,0.77)",0.000
std_read_news_books,6094,"0.81(0.67,0.98)",0.031
std_majong_or_cards,1873,"0.59(0.44,0.79)",0.000
std_tv_or_radio,18579,"0.37(0.32,0.42)",0.000
current_CI_score,3,"0.91(0.90,0.91)",0.000
education_1.0,7938,"0.71(0.61,0.82)",0.000
education_2.0,3115,"0.69(0.53,0.89)",0.004


In [34]:
output3.loc[list(set(output3.index)-set(output1.Variable)),:].sort_values(by = 'Variable')

,Num of samples,OR,pvalues
Variable,,,
majong_or_cards_diff_-1.0,3341,"1.29(1.08,1.53)",0.005
majong_or_cards_diff_1.0,1499,"0.58(0.42,0.81)",0.001
majong_or_cards_diff_2.0,1129,"0.37(0.24,0.56)",0.000
plant_flower_bird_diff_-1.0,3315,"1.44(1.11,1.87)",0.006
plant_flower_bird_diff_1.0,1003,"0.61(0.43,0.87)",0.006
plant_flower_bird_diff_2.0,2196,"0.47(0.36,0.62)",0.000
tv_or_radio_diff_-1.0,5503,"2.51(2.19,2.87)",0.000
tv_or_radio_diff_1.0,2569,"0.91(0.76,1.09)",0.314
tv_or_radio_diff_2.0,2241,"0.52(0.44,0.63)",0.000


In [35]:
total_df_for_logit.tv_or_radio_diff.value_counts()

0.000     12606
-1.000     1820
1.000      1788
-4.000     1638
4.000      1288
-3.000     1149
3.000       953
-2.000      896
2.000       781
5.000         1
Name: tv_or_radio_diff, dtype: int64